In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
from matplotlib.animation import FuncAnimation

In [ ]:
n = 100     #formerly grid_size
grid = np.zeros([n, n], dtype=int)

separation_factor = 2/3     # horizontal separation of the plates
vertical_aspect = 0.8   # amount of vertical grid_size taken up by plate

y1 = int(0.5*n*(1 - vertical_aspect))
y2 = int(0.5*n*(1 + vertical_aspect))
assert 0 <= separation_factor <= 1
assert 0 <= vertical_aspect <= 1
offset = int(separation_factor * (n//2))
grid[y1:y2, offset] = 1
grid[y1:y2, n-offset-1] = -1
plt.imshow(grid.T, cmap='viridis')
plt.show()

In [ ]:
diags = np.array([np.ones(n*n) * -4, np.ones(n*n), np.ones(n*n), np.ones(n*n), np.ones(n*n)])
laplacian = sps.spdiags(diags, [0, 1, -1, n, -n], n*n, n*n)
b = sps.csr_matrix(grid.flatten().astype(float)).reshape(-1, 1) # FAST sparsemat conversion
A = laplacian.copy().tocsc()
x = sps.linalg.spsolve(A, b) # solving the laplacian

In [ ]:
potential = x.reshape(100, 100)
levels_pos = np.array([0.1, 0.2, 0.4, 0.6, 0.8, 1]) # determine levels to be shown for each "eye" on the contour plot
levels_neg = np.flip(levels_pos) * -1
levels = np.hstack([levels_neg, levels_pos]) * potential.max()

In [ ]:
plt.contour(potential.T, cmap='viridis', levels=levels)
plt.title('Potential Plot')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

In [ ]:
def get_col_gradient(arr):
    assert arr.ndim == 2
    n = len(arr.T)
    grad = (np.pad(arr, ((0, 0), (0, 2))) - np.pad(arr, ((0, 0), (2, 0))))/2
    grad = grad[:, 1:n+1]
    fw_diff = arr[:, 1] - arr[:, 0]
    bw_diff = arr[:, n-1] - arr[:, n-2]
    grad[:, 0] = fw_diff
    grad[:, n-1] = bw_diff
    return grad

def get_row_gradient(arr):
    assert arr.ndim == 2
    n = len(arr)  # Now working with rows instead of columns
    grad = (np.pad(arr, ((0, 2), (0, 0))) - np.pad(arr, ((2, 0), (0, 0))))/2
    grad = grad[1:n+1, :]
    fw_diff = arr[1, :] - arr[0, :]
    bw_diff = arr[n-1, :] - arr[n-2, :]
    grad[0, :] = fw_diff
    grad[n-1, :] = bw_diff
    return grad

def gradient(arr):
    return (get_row_gradient(arr), get_col_gradient(arr))

field = gradient(potential)

In [ ]:
class Particle:
    def __init__(self):
        self.x = np.random.normal(n/2, 10)
        self.y = np.random.normal(n/2, 10)
        self.Ux = field[0][int(self.x), int(self.y)]
        self.Uy = field[1][int(self.x), int(self.y)]

    def move(self):
        self.x += self.Ux
        self.y += self.Uy

    def plot(self, axesObject):
        axesObject.plot(self.x, self.y, color='yellow', marker='o')

    def quiver(self, axesObject):
        axesObject.quiver(self.x, self.y, self.Ux, self.Uy, color='yellow')


electron = Particle()


In [ ]:
fig, ax = plt.subplots()

ax.contour(potential.T, cmap='viridis', levels=levels) # static contour plot
ax.set_title('Particle Plot')
ax.set_xlabel('X')
ax.set_ylabel('Y')

particle_group = [Particle() for i in range(12)]
def update(axesObject, particle_group):
    for particle in particle_group:
        particle.move()
    return particle_group


